![GPU Basics Slide](img/02_GPU_Basics/Folie1.PNG)

![GPU Basics Slide](img/02_GPU_Basics/Folie2.PNG)

![GPU Basics Slide](img/02_GPU_Basics/Folie3.PNG)

In [2]:
import warnings

import numpy as np
from numba import cuda, float32, config, vectorize
from numba.core.errors import NumbaPerformanceWarning

config.CUDA_ENABLE_PYNVJITLINK = 1
warnings.filterwarnings("ignore", category=NumbaPerformanceWarning)

In [ ]:
!mkdir profiling

Check if we have a CUDA device available.

In [ ]:
# Check if CUDA is available
print(cuda.gpus)

<Managed Device <CUdevice 0>>


Now let's implement GPU computation using the `@vectorize` decorator with the target set to 'cuda'. This will automatically execute the code on the GPU.

In [4]:
vectorA = np.random.rand(65536).astype(np.float32)
vectorB = np.random.rand(65536).astype(np.float32)

@vectorize([float32(float32, float32)], target='cuda')
def squared_error(d_vectorA, d_vectorB):
    return (d_vectorA - d_vectorB) ** 2

vectorC = squared_error(vectorA, vectorB)
print(f"We get results from the GPU: {vectorC[0:5]}")

We get results from the GPU: [3.5241649e-01 1.0848190e-04 1.0519932e-02 7.4898817e-02 1.6896421e-01]


What happened exactly? How does this work under the hood? Let's have a closer look.

![GPU Basics Slide](img/02_GPU_Basics/Folie4.PNG)

![GPU Basics Slide](img/02_GPU_Basics/Folie5.PNG)

![GPU Basics Slide](img/02_GPU_Basics/Folie6.PNG)

![GPU Basics Slide](img/02_GPU_Basics/Folie7.PNG)

![GPU Basics Slide](img/02_GPU_Basics/Folie8.PNG)

![GPU Basics Slide](img/02_GPU_Basics/Folie9.PNG)

![GPU Basics Slide](img/02_GPU_Basics/Folie10.PNG)

### Task 0

To analyze profiling results, please install the NVIDIA tools on your **local machine**.

> **Note:** You do **not** need an NVIDIA GPU on your computer.  
> The profiling itself runs on the server. You only need the tools locally to open and inspect the result files in the GUI.

- [Nsight Systems (Nsys)](https://developer.nvidia.com/nsight-systems/get-started)
- [Nsight Compute (Ncu)](https://developer.nvidia.com/tools-overview/nsight-compute/get-started)

Now let us use the NVIDIA Nsight Systems (nsys) profiling tool to examine the timeline shown in the slide above.

We need to isolate our code into a Python file to execute it easily with nsys:

In [ ]:
%%writefile profiling/cuda_vectorize.py
import numpy as np
from numba import vectorize, float32, cuda

vectorA = np.random.rand(65536).astype(np.float32)
vectorB = np.random.rand(65536).astype(np.float32)

@vectorize([float32(float32, float32)], target='cuda')
def squared_error(d_vectorA, d_vectorB):
    return (d_vectorA - d_vectorB) ** 2

vectorC = squared_error(vectorA, vectorB)
print(f"We get results from the GPU: {vectorC[0:5]}")

Overwriting profiling/cuda_vectorize.py


We can wrap that Python file with the nsys profiler (which is preinstalled in this environment) to analyze the execution timeline and understand the code's performance characteristics.

In [3]:
!nsys profile -f true -o profiling/cuda_vectorize -t cuda --stats true python3 profiling/cuda_vectorize.py

Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.



/home/azureuser/src/HS25/.venv/lib/python3.12/site-packages/numba_cuda/numba/cuda/dispatcher.py:686: NumbaPerformanceWarning: Grid size 64 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
We do actually get results from teh GPU: [0.5192787  0.03461697 0.03134166 0.5295557  0.61284894]
Generating '/tmp/nsys-report-9c5e.qdstrm'
[1/6] [========================100%] cuda_vectorize.nsys-rep
[2/6] [========================100%] cuda_vectorize.sqlite
[3/6] Executing 'cuda_api_sum' stats report

 Time (%)  Total Time (ns)  Num Calls  Avg (ns)  Med (ns)  Min (ns)  Max (ns)  StdDev (ns)         Name       
 --------  ---------------  ---------  --------  --------  --------  --------  -----------  ------------------
     47.9           321914          3  107304.7    6323.0      5210    310381     175870.1  cuMemAlloc_v2     
     15.0           100713          2   50356.5   50356.5     36240     64473      19963.7  cuMemcpyHtoD_v2   
     11.3

In fact, we do see the same timeline as in the slides. The big gap between the H2D and the kernel launch is due to
* Python is slow in general
* The function needs to be compiled first.

![Nsys profiling of vectorized CUDA kernel](img/01_vectorize_nsys.png)

### Task 1: JIT impact

Modify the `cuda_vectorize.py` file such that the function is called multiple times. Does the big gap disappear?

Let us dig deeper. What exactly happens on the GPU? How is the calculation distributed among the thousands of cores a GPU has? How can we control this ourselves?

![GPU Basics Slide](img/02_GPU_Basics/Folie11.PNG)

![GPU Basics Slide](img/02_GPU_Basics/Folie12.PNG)

![GPU Basics Slide](img/02_GPU_Basics/Folie13.PNG)

![GPU Basics Slide](img/02_GPU_Basics/Folie14.PNG)

![GPU Basics Slide](img/02_GPU_Basics/Folie15.PNG)

![GPU Basics Slide](img/02_GPU_Basics/Folie16.PNG)

### Task 2: Understand the CUDA Execution Model

Modify the `hello_kernel` below to launch at least 8 different kernel configurations. Manually calculate the global thread ID inside the kernel. Experiment with both 1D and 2D grid/block layouts. For each configuration, consider what happens on the GPU and how the execution differs on the hardware. Document your observations and reasoning for each configuration.

In [ ]:
@cuda.jit
def hello_kernel():
    # ToDo: calculate the global thread ID manually
    global_id = cuda.grid(1)
    print(
        "Hello from thread", global_id, 
        "- I'm in block", cuda.blockIdx.x, 
        "of size", cuda.blockDim.x, 
        "and local threadIdx", cuda.threadIdx.x
    )

# ToDo: test a lot of different configurations and understand what is happening on the GPU
hello_kernel[4, 4]()

Hello from thread 4 - I'm in block 1 of size 4 and local threadIdx 0
Hello from thread 5 - I'm in block 1 of size 4 and local threadIdx 1
Hello from thread 6 - I'm in block 1 of size 4 and local threadIdx 2
Hello from thread 7 - I'm in block 1 of size 4 and local threadIdx 3
Hello from thread 0 - I'm in block 0 of size 4 and local threadIdx 0
Hello from thread 1 - I'm in block 0 of size 4 and local threadIdx 1
Hello from thread 2 - I'm in block 0 of size 4 and local threadIdx 2
Hello from thread 3 - I'm in block 0 of size 4 and local threadIdx 3
Hello from thread 8 - I'm in block 2 of size 4 and local threadIdx 0
Hello from thread 9 - I'm in block 2 of size 4 and local threadIdx 1
Hello from thread 10 - I'm in block 2 of size 4 and local threadIdx 2
Hello from thread 11 - I'm in block 2 of size 4 and local threadIdx 3
Hello from thread 12 - I'm in block 3 of size 4 and local threadIdx 0
Hello from thread 13 - I'm in block 3 of size 4 and local threadIdx 1
Hello from thread 14 - I'm in 

ToDo: write down findings

### Task 3: Write a numba.cuda kernel with explicit calls

Now it is your turn. Write the `squared_error_kernel` code with explicit data transfers and allocations. Try out different kernel launch parameters and think about which ones make sense. Once it works, profile it using nsys. Do you see the exact same behavior as it the vectorized version above?

In [ ]:
#ToDo

[0.03245101 0.00085454 0.02855932 0.01089774 0.0108772  0.07373396]
